In [1]:
!pip install -q beautifulsoup4==4.12.2 PyMuPDFb==1.24.10 PyMuPDF==1.24.13 tqdm==4.66.5 duckdb==0.10.0 tabulate==0.9.0 plotly==5.20.0 python-dotenv openai==1.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 16.5 MB/s eta 0:00:00


In [2]:
SYMBOL = "SCC"

In [19]:
import requests
import pandas as pd
import re



session = requests.Session()
session.get(f"https://www.settrade.com/th/equities/quote/{SYMBOL}/financial-statement/full")


headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-TH,en;q=0.9,th-TH;q=0.8,th;q=0.7,en-GB;q=0.6,en-US;q=0.5',
    # 'cookie': 'visid_incap_2685215=l/JAPqmmR8KSk7rLhs3BzKvPSGYAAAAAQUIPAAAAAABALhVFFEmAe3vRYqQgqqQo; exp_history={"go_expid":"5AD93i4KR9-ZVNOhL9Vr2w-V2","msgt":"popup","count":1}; _cc_id=4d4cfaa1a003693b7c5cc45baf0913e9; __gads=ID=f9d48026689a99be:T=1716047793:RT=1716047793:S=ALNI_Ma_zJhrDoYFiXyfutD1XSHxPWA3Uw; __gpi=UID=00000e2256455e78:T=1716047793:RT=1716047793:S=ALNI_Mby0wdaHRd_h-eJjYJY5wLHS0TJAQ; __eoi=ID=2567fce2f1c28622:T=1716047793:RT=1716047793:S=AA-AfjaTaPE5D-KD9jKsPW8xY0gE; cto_bundle=44weE19pQlBrWWZRQUdySnclMkJmNXlkNkk4blI2MTgwTGNzSUZGSSUyRjRjdGRsUUZMZ1pWSDViOHIlMkY1c3lHc0NuZjBReDUlMkZVd21FT3piZkJ0MmE3bEEyRCUyQlVqYlFWbWpGVkQ0S1gxMlZUQTdIZXV3T05iN2VqbGZKWlNqRnloTm5jT3ltTnBCS1kzc3lxVWFUVEcxWFhHd2FnUjdCb1VMRWFzcmJKaG1SWm0lMkZqJTJGcE03c0UwJTJGYXRHU3FpUiUyQlhSRWlhMkxNd2o; cto_bidid=b-mYHF9lQlh6WHpHZHNta0hieDFuZ21FQ2dKQjdWRnR2RHNydllBQ2hkZEhOb1RxRzFhM1hGU29zbDgxcVdHOTFCQm1jJTJCb0ZaOVdGdjBPcCUyQnVmTGVVTUJHUmtCNDVkNTBOM1ZMdG9sVWg2OER1TFZDaENRNFElMkZjTjYwaFpGY0lsV3hjUUozZ2NUJTJGRG1KWWl5QlNqWWE5TVlKUSUzRCUzRA; visid_incap_2868064=9TnELvraSEyQ3rCeC/3tp6/imGYAAAAAQUIPAAAAAABQXj/9u+lgqIuVKksYLcer; _ga_W7C0TNQT63=GS1.1.1721377038.3.1.1721377082.0.0.0; charlot=6a8621f0-cdcf-45a6-b1c2-f81785e2d02f; _gcl_au=1.1.1685208698.1724255273; _gid=GA1.2.1052211754.1724255275; api_call_counter=5; SET_COOKIE_POLICY=20231111093657; nlbi_2685215=IC0uOhgeyBy9J7BzwZdY4QAAAAAqEQh9oBWiKDUUC927BZYM; _cbclose=1; _cbclose64035=1; route=0f35441e6abd8a06b9ca7c7f54fa2b5e; incap_ses_235_2685215=SjIpB/f0VXZxsG4/XeNCAyHexmYAAAAA4/zIE2DbjM+Hy9alp/2MXA==; landing_url=https://www.settrade.com/th/equities/quote/OR/overview?_gl=1%2a9iyx2w%2a_gcl_au%2aMTYwODA1NzI0NS4xNzIzNjQzNjg2%2a_ga%2aMTgyMzgyMTUxNS4xNzE1NTk1MjY5%2a_ga_ET2H60H2CB%2aMTcyNDI1NTIxMS4yNi4xLjE3MjQyNTU0ODguNDkuMC4w; _uid64035=DB23855A.7; _ctout64035=1; _gat_UA-29750633-11=1; _ga=GA1.1.1767658003.1716047789; _ga_W11E901KXL=GS1.1.1724324251.9.1.1724324280.31.0.0; _ga_6WS2P0P25V=GS1.1.1724324251.9.1.1724324280.31.0.0',
    'if-none-match': '"342c523b"',
    'priority': 'u=1, i',
    'referer': 'https://www.settrade.com/th/equities/quote/OR/financial-statement/full',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

def get_sheet(period: str) -> pd.DataFrame:
    params = {
        'accountType': 'balance_sheet',
        'fsType': 'company',
        'period': period,
#         'language': 'th',
        'language': 'ENG',
    }

    response = session.get(
        f'https://www.settrade.com/api/set/stock/{SYMBOL}/financialstatement',
        params=params,
        headers=headers,
    )
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data["accounts"])

def get_business_type(symbol: str) -> str:
    response = session.get(f"https://www.settrade.com/th/equities/quote/{symbol}/overview")

    response.raise_for_status()
    text = response.text
    pattern = r'businessType:\s*"([^"]+)"'
    found = re.search(pattern, text)

    if found:
        target_data = found.group(1)
        return target_data
    else:
        print("No match found.")
        return None
    return response.json()

In [21]:
from_ = 2000
to = 2024

year_range = range(from_, to)
first_year, *year_range = year_range

In [39]:

sheet = get_sheet(f"YE_{first_year}")[["accountCode", "accountName", "amount"]]
sheet[str(first_year)] = sheet["amount"]
sheet = sheet.drop("amount", axis=1)
for year in year_range:
    sheet_by_year = get_sheet(f"YE_{year}")[["accountCode", "accountName", "amount"]]
    sheet_by_year[str(year)] = sheet_by_year["amount"]
    sheet_by_year = sheet_by_year.drop("amount", axis=1)

    sheet = pd.merge(sheet, sheet_by_year, on="accountCode", how='outer', suffixes=("", f"_{year}"))
    sheet['accountName'] = sheet['accountName'].fillna(sheet[f'accountName_{year}'])
    sheet = sheet.drop(f'accountName_{year}', axis=1)

sheet


,accountCode,accountName,2000,2001,2002,2003,2004,2005,2006,2007,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,100000,สินทรัพย์,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109999,รวมสินทรัพย์,196144720.0,181682714.0,182618838.0,203084480.0,218845795.0,156713724.0,167185412.0,130967023.0,...,248113107.0,264282399.0,271633132.0,286650904.0,289921613.0,282990658.0,292305183.0,359824537.0,344715814.0,346280041.0
2,110000,สินทรัพย์หมุนเวียน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110100,เงินสดและรายการเทียบเท่าเงินสด,1079947.0,3859361.0,878869.0,154240.0,748585.0,191939.0,223910.0,1199667.0,...,6543121.0,4794733.0,15893804.0,32035244.0,14128414.0,11691164.0,10916024.0,5342977.0,22211969.0,10878668.0
4,111600,เงินลงทุนระยะสั้น - สุทธิ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7969790.0,16284158.0,9833853.0,9428604.0,15271317.0,9437013.0,26925648.0,1231620.0,19279307.0,3619466.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,319921,ส่วนเกิน (ต่ำกว่า) ทุนจากการเปลี่ยนแปลงส่วนได้...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1446580.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,319950,ผลต่างอัตราแลกเปลี่ยนจากการแปลงค่างบการเงิน,31720.0,NaN,NaN,-650505.0,-735693.0,-441109.0,-1031706.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,319990,องค์ประกอบอื่นของส่วนของผู้ถือหุ้น - อื่นๆ,NaN,29148888.0,47651710.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,31449.0,NaN,7182799.0,5804497.0,5997298.0
82,319999,รวมส่วนของผู้ถือหุ้นของบริษัทใหญ่,53832497.0,57299945.0,75613022.0,104966253.0,129130289.0,63947043.0,75022951.0,35173047.0,...,86561994.0,88928117.0,93932076.0,94145835.0,95681454.0,96691732.0,106277526.0,182693811.0,193147443.0,197457428.0


In [41]:
sheet["accountName"].unique()

array(['สินทรัพย์', 'รวมสินทรัพย์', 'สินทรัพย์หมุนเวียน',
       'เงินสดและรายการเทียบเท่าเงินสด', 'เงินลงทุนระยะสั้น - สุทธิ',
       'ลูกหนี้การค้าและลูกหนี้หมุนเวียนอื่น - สุทธิ',
       'บุคคลหรือกิจการอื่น', 'บุคคลหรือกิจการที่เกี่ยวข้องกัน',
       'เงินให้กู้ยืมระยะสั้นและดอกเบี้ยค้างรับ', 'ลูกหนี้หมุนเวียนอื่น',
       'สินทรัพย์ไม่หมุนเวียน และ/หรือ กลุ่มสินทรัพย์ที่ยกเลิกซึ่งถือไว้เพื่อขาย',
       'สินทรัพย์หมุนเวียนอื่น', 'สินทรัพย์หมุนเวียนอื่น - อื่น ๆ',
       'รวมสินทรัพย์หมุนเวียน', 'สินทรัพย์ไม่หมุนเวียน',
       'เงินลงทุนระยะยาว - สุทธิ',
       'เงินให้กู้ยืมระยะยาวและดอกเบี้ยค้างรับ - ส่วนที่ถึงกำหนดชำระเกินกว่าหนึ่งปี',
       'เงินลงทุนในบริษัทย่อย บริษัทร่วม และการร่วมค้า ที่บันทึกโดยวิธีส่วนได้เสีย - สุทธิ',
       'เงินลงทุนในบริษัทย่อย บริษัทร่วม และการร่วมค้า ที่บันทึกด้วยวิธีอื่น - สุทธิ',
       'สินทรัพย์ทางการเงินไม่หมุนเวียนอื่น',
       'สินทรัพย์ทางการเงินไม่หมุนเวียนอื่น - อื่น ๆ',
       'อสังหาริมทรัพย์เพื่อการลงทุน - สุทธิ',
       'ที่ดิน อาคาร 

In [42]:
business_type = get_business_type(SYMBOL)

Extracted data: ธุรกิจการลงทุน (Holding company) ใน 3 ธุรกิจหลัก ได้แก่ ธุรกิจซีเมนต์และผลิตภัณฑ์ก่อสร้าง ธุรกิจเคมิคอลส์ และธุรกิจแพคเกจจิ้ง


In [51]:
from string import Template


template = Template("""
# Context:

Stock name: ${SYMBOL}
Business type: ${business_type}

Balance sheet: ${sheet}
Structure:

${task}
""")
sheet_prompt = sheet.drop("accountCode", axis=1).astype(str).to_markdown(index=False, floatfmt=",.1f")
print(
    template.safe_substitute(
        SYMBOL=SYMBOL,
        business_type=business_type,
        sheet=sheet_prompt,
    )
)


# Context:

Stock name: SCC
Business type: ธุรกิจการลงทุน (Holding company) ใน 3 ธุรกิจหลัก ได้แก่ ธุรกิจซีเมนต์และผลิตภัณฑ์ก่อสร้าง ธุรกิจเคมิคอลส์ และธุรกิจแพคเกจจิ้ง

Balance sheet: | accountName                                                      |          2000 |          2001 |          2002 |          2003 |          2004 |          2005 |          2006 |          2007 |          2008 |          2009 |          2010 |          2011 |          2012 |          2013 |          2014 |          2015 |          2016 |          2017 |          2018 |          2019 |          2020 |          2021 |          2022 |          2023 |
|:-----------------------------------------------------------------|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|-----

# Set up LLM

In [52]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
from typing_extensions import TypedDict
from IPython.display import Markdown
from google.colab import userdata


GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)


MODEL = "typhoon-v1.5x-70b-instruct"



TYPHOON_API_KEY = userdata.get('TYPHOON_API_KEY')
MODEL = "typhoon-v1.5x-70b-instruct"
client = OpenAI(
   api_key=TYPHOON_API_KEY,
   base_url='https://api.opentyphoon.ai/v1'
)


class ChatTurn(TypedDict):
    role: str
    content: str


def get_open_ai_completion(
    prompt: str,
    model: str = MODEL,
    stream: bool = False,
    initial_message: list[ChatTurn]| None = None,
    temperature: float = 0
) -> str:
    if initial_message is None:
        initial_message = []
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=temperature, stream=stream
    )
    if not stream:
        return response.choices[0].message.content
    else:
        result = ""
        for chunk in response:
            content = chunk.choices[0].delta.content
            print(content, end="")
            if isinstance(content, str):
                result += content
        return result


def get_gemini_completion(
    prompt: str,
    stream: bool = False,
) -> str:
    # model = genai.GenerativeModel("gemini-1.5-pro")
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

In [56]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="Analyse this stock balance sheet in Thai"
)
Markdown(get_gemini_completion(prompt))

## วิเคราะห์งบดุลของบริษัท SCC (Holding Company)

**SCC** เป็นบริษัทโฮลดิ้งที่ดำเนินธุรกิจการลงทุนใน 3 กลุ่มธุรกิจหลัก ได้แก่  ธุรกิจซีเมนต์และผลิตภัณฑ์ก่อสร้าง ธุรกิจเคมิคอลส์ และธุรกิจแพคเกจจิ้ง 

**สินทรัพย์**:

* **สินทรัพย์รวม**: มูลค่าของสินทรัพย์ SCC เพิ่มขึ้นอย่างต่อเนื่องตลอดช่วงเวลาที่ผ่านมา  โดยเฉพาะอย่างยิ่งในปี 2021 และ 2022  ซึ่งบ่งบอกถึงการลงทุนและการขยายธุรกิจอย่างต่อเนื่อง
* **สินทรัพย์หมุนเวียน**: ประกอบด้วยเงินสดและรายการเทียบเท่าเงินสด เงินลงทุนระยะสั้น  ลูกหนี้การค้า  และเงินให้กู้ยืมระยะสั้น ซึ่งเป็นสินทรัพย์ที่มีสภาพคล่องสูง   ในปี 2022  SCC มีสภาพคล่องที่ดี  แสดงให้เห็นถึงความสามารถในการชำระหนี้ในระยะสั้นได้
* **สินทรัพย์ไม่หมุนเวียน**:   ประกอบด้วยเงินลงทุนระยะยาว เงินให้กู้ยืมระยะยาว  เงินลงทุนในบริษัทย่อยและบริษัทร่วม  และสินทรัพย์ทางการเงินไม่หมุนเวียนอื่น   โดยส่วนใหญ่เป็นการลงทุนในธุรกิจหลักของ SCC  แสดงถึงการลงทุนเพื่อการเติบโตในระยะยาว

**หนี้สิน**:

* **หนี้สินรวม**:  เพิ่มขึ้นในช่วงปี 2016-2020 และลดลงในช่วงปี 2021-2023  อาจเกี่ยวข้องกับการลงทุนและการกู้ยืมเงินเพื่อขยายธุรกิจ 
* **หนี้สินหมุนเวียน**:   ประกอบด้วยเงินเบิกเกินบัญชี  เจ้าหนี้การค้า  เงินกู้ยืมระยะสั้น และส่วนของหนี้สินระยะยาวส่วนที่ถึงกำหนดชำระภายในหนึ่งปี  ซึ่งเป็นหนี้สินที่มีความสำคัญต่อความสามารถในการชำระหนี้ระยะสั้น  SCC มีหนี้สินหมุนเวียนที่ค่อนข้างสูง  อาจส่งผลต่อความเสี่ยงในการชำระหนี้ 
* **หนี้สินไม่หมุนเวียน**:   ประกอบด้วยหนี้สินระยะยาว  และหนี้สินตามสัญญาเช่า   SCC มีหนี้สินไม่หมุนเวียนสูง  บ่งบอกถึงการกู้ยืมเงินเพื่อการลงทุนในระยะยาว

**ส่วนของผู้ถือหุ้น**:

* **ส่วนของผู้ถือหุ้นรวม**:  เพิ่มขึ้นอย่างต่อเนื่อง   ซึ่งสะท้อนถึงผลประกอบการที่ดีของ SCC 
* **ทุนจดทะเบียน**:   คงที่ตลอดช่วงเวลา  แสดงถึงโครงสร้างทุนของ SCC ที่มั่นคง
* **กำไร (ขาดทุน) สะสม**:   SCC มีกำไรสะสมที่เพิ่มขึ้นอย่างต่อเนื่อง  บ่งบอกถึงผลประกอบการที่ดีของบริษัท   
* **องค์ประกอบอื่นของส่วนของผู้ถือหุ้น**:  มีการเปลี่ยนแปลงอย่างมากในแต่ละปี   แสดงถึงการจัดสรรเงินทุนและการปรับโครงสร้างทุนอย่างต่อเนื่อง

**สรุป**:

* SCC มีสินทรัพย์รวมที่เพิ่มขึ้น  บ่งบอกถึงการลงทุนและการเติบโตอย่างต่อเนื่อง
* SCC มีหนี้สินรวมที่ค่อนข้างสูง  โดยเฉพาะหนี้สินหมุนเวียน   ซึ่งอาจส่งผลต่อความเสี่ยงในการชำระหนี้
* SCC มีส่วนของผู้ถือหุ้นที่เพิ่มขึ้น  สะท้อนถึงผลประกอบการที่ดีของบริษัท

**หมายเหตุ**:

*  การวิเคราะห์งบดุลควรทำร่วมกับงบกำไรขาดทุนและงบกระแสเงินสด  เพื่อให้ได้ภาพรวมของสถานะทางการเงินของ SCC อย่างครบถ้วน
*  ควรพิจารณาปัจจัยอื่น ๆ เช่น  สภาพเศรษฐกิจ  อุตสาหกรรม  และการแข่งขัน  ในการวิเคราะห์งบดุลของ SCC 


In [ ]:
import matplotlib
import matplotlib.font_manager as fm
from os import path
if not path.exists('Sarabun-Regular.ttf'):
    !wget https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf -O Sarabun-Regular.ttf

if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

--2024-11-05 10:43:28--  https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google/fonts/main/ofl/sarabun/Sarabun-Regular.ttf [following]
--2024-11-05 10:43:29--  https://raw.githubusercontent.com/google/fonts/main/ofl/sarabun/Sarabun-Regular.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90220 (88K) [application/octet-stream]
Saving to: ‘Sarabun-Regular.ttf’

Sarabun-Regular.ttf 100%[===================>]  88.11K  --.-KB/s    in 0.009s  

2024-11-05 10:43:29 (9.88 MB/s) - ‘Sarabun-Regular.ttf’ saved [90220/90220]



In [57]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought how this data can support the investors
2. Use Matplotlib to create some visualisations to ease investors. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix."""
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)

SyntaxError: '[' was never closed (<string>, line 816)

In [58]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Comment file header and show your thought about income structure of this company.
2. Use Plotly to create some visualisations to ease investors. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix."""
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)

SyntaxError: '[' was never closed (<string>, line 179)

In [60]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    task="""Act as finanical analyst and analyse this data and write a Python code with the following sections:

1. Thought about income structure 2022 vs 2023 and write your in comment section
2. Use Plotly to create some visualisations to ease investors focusing on 2022 vs 2023. Note:
 - You need to rotate X-axis ticks with 45 degree.
 - plot legends should be outside the chartz
3. Use a string variable named "conclusion" to store your data analytics and conclusion in Markdown format in Thai language.

You must give only the answer do not add prologue, prefix and suffix."""
)

python_code = get_gemini_completion(prompt)
exec(python_code.strip("`").lstrip("python").strip())
Markdown(conclusion)


##  สรุปผลการวิเคราะห์ข้อมูลงบดุลของ SCC 

จากการวิเคราะห์ข้อมูลงบดุลของ SCC ประจำปี 2022 และ 2023 พบว่า 

* **สินทรัพย์**: ในปี 2023 สินทรัพย์ของ SCC มีมูลค่าเพิ่มขึ้นเล็กน้อยเมื่อเทียบกับปี 2022 โดยสินทรัพย์หมุนเวียนและสินทรัพย์ไม่หมุนเวียนมีการเติบโตอย่างต่อเนื่อง
* **หนี้สิน**: สมมติว่าหนี้สินทั้งหมดไม่มีการเปลี่ยนแปลงในปี 2023
* **ส่วนของผู้ถือหุ้น**: สมมติว่าส่วนของผู้ถือหุ้นทั้งหมดไม่มีการเปลี่ยนแปลงในปี 2023

อย่างไรก็ตาม ข้อมูลงบดุลเป็นเพียงข้อมูลเบื้องต้นเกี่ยวกับสถานะทางการเงินของ SCC  
การวิเคราะห์เชิงลึกจำเป็นต้องพิจารณาข้อมูลอื่นๆ เช่น งบกำไรขาดทุน และงบกระแสเงินสด 
เพื่อให้ได้ข้อสรุปที่ชัดเจนเกี่ยวกับสถานะทางการเงินและแนวโน้มการเติบโตของ SCC 


In [62]:
prompt = template.safe_substitute(
    SYMBOL=SYMBOL,
    sheet=sheet_prompt,
    business_type=business_type,
    task="""
# Objective:
Answer the following questions:

2 วิเคราะห์หนี้สิน
2.1 ดูโครงสร้างหข้องหนี้สินว่าสอดคล้องกับโครงสร้างสินทรัพย์หรือไม่
2.1.1ถ้าบริษัทที่มีสินทรัพย์หมุนเวียนมากกว่าสินทรัพย์ไม่หมุนเวียน หนี้สินหมุนเวียนจะมากกว่าหนี้สินไม่หมุนเวียน แสดงว่าโครงสร้างหนี้สินสอดคล้องกับสินทรัพย์
2.1.2ถ้าบริษัทที่มีสินทรัพย์หมุนเวียนน้อยกว่าสินทรัพย์ไม่หมุนเวียน หนี้สินหมุนเวียนจะน้อยกว่าหนี้สินไม่หมุนเวียน แสดงว่าโครงสร้างหนี้สินสอดคล้องกับสินทรัพย์
2.1.3 ถ้าบริษัทมีสินทรัพย์ไม่หมุนเวียนเยอะ แต่หนี้ส่วนใหญ่เป็นหนี้ระยะสั้น ให้ระวังปัญหาหนี้สิน เนื่องจากใช้เงินผิดประเภทกู้ระยะสั้นมาลงทุนระยะยาว
2.2 ให้ระวังบริษัทที่มีส่วนของหนี้สินเพิ่มขึ้นเร็วกว่าส่วนของผู้ถือหุ้นที่เพิ่มขึ้น อาจมีปัญหาหนี้สินได้ในอนาคต
2.3 วิเคราะห์ส่วนของผู้ถือหุ้น
2.3.1 ให้ระวังบริษัทที่ขาดทุนสะสมจำนวนมากเมื่อเทียบกับทุนจดทะเบียน แสดงว่าในอดีตที่ผ่านมา มีผลประกอบการขาดทุนเป็นจำนวนมาก
2.3.2 ให้ระวังบริษัทที่มี ตราสารหนี้เสมือนทุน จำนวนมากเมื่อเทียบกับส่วนของผู้ถือหุ้น เพราะจริงๆแล้วเป็นหนี้สินไม่ใช่ส่วนของผู้ถือหุ้น

Give the answer as bullet points


# Style:
Business analyst expert

# Audience:
The Thai CEO who want to understand the company financial structure

# Response:
Markdown bulletpoints with Thai language and frame into a table.

for example:
2.1 ดูโครงสร้างหข้องหนี้สินว่าสอดคล้องกับโครงสร้างสินทรัพย์หรือไม่:

<Your answer with your reason>

2.2 ให้ระวังบริษัทที่มีส่วนของหนี้สินเพิ่มขึ้นเร็วกว่าส่วนของผู้ถือหุ้นที่เพิ่มขึ้น อาจมีปัญหาหนี้สินได้ในอนาคต:

<Your answer with your reason>
"""
)

output = get_gemini_completion(prompt)
Markdown(output)

##  วิเคราะห์โครงสร้างทางการเงินของบริษัท SCC

| ข้อ | คำถาม | คำตอบ | เหตุผล |
|---|---|---|---|
| 2.1 | ดูโครงสร้างหข้องหนี้สินว่าสอดคล้องกับโครงสร้างสินทรัพย์หรือไม่ | โครงสร้างหนี้สินของ SCC สอดคล้องกับโครงสร้างสินทรัพย์ | บริษัท SCC มีสินทรัพย์ไม่หมุนเวียนมากกว่าสินทรัพย์หมุนเวียน และหนี้สินไม่หมุนเวียนก็มากกว่าหนี้สินหมุนเวียนเช่นกัน  |
| 2.2 | ให้ระวังบริษัทที่มีส่วนของหนี้สินเพิ่มขึ้นเร็วกว่าส่วนของผู้ถือหุ้นที่เพิ่มขึ้น อาจมีปัญหาหนี้สินได้ในอนาคต | SCC มีสัญญาณบ่งบอกถึงปัญหาหนี้สิน | ส่วนของหนี้สินของ SCC เพิ่มขึ้นอย่างต่อเนื่องในช่วงหลายปีที่ผ่านมา ขณะที่ส่วนของผู้ถือหุ้นเพิ่มขึ้นช้ากว่ามาก แสดงให้เห็นว่าบริษัทพึ่งพาการกู้ยืมเป็นหลักในการขยายกิจการ ซึ่งอาจนำไปสู่ปัญหาหนี้สินในอนาคต |
| 2.3.1 | ให้ระวังบริษัทที่ขาดทุนสะสมจำนวนมากเมื่อเทียบกับทุนจดทะเบียน แสดงว่าในอดีตที่ผ่านมา มีผลประกอบการขาดทุนเป็นจำนวนมาก | SCC มีผลประกอบการที่มั่นคง | แม้ว่า SCC จะมีกำไรสะสมที่เพิ่มขึ้นอย่างต่อเนื่อง แต่ในอดีตที่ผ่านมา บริษัทมีการขาดทุนสะสมอยู่บ้าง แสดงให้เห็นถึงความผันผวนของผลประกอบการ |
| 2.3.2 | ให้ระวังบริษัทที่มี ตราสารหนี้เสมือนทุน จำนวนมากเมื่อเทียบกับส่วนของผู้ถือหุ้น เพราะจริงๆแล้วเป็นหนี้สินไม่ใช่ส่วนของผู้ถือหุ้น | SCC ไม่มีตราสารหนี้เสมือนทุน | บริษัท SCC มีโครงสร้างทางการเงินที่เรียบง่ายและไม่ซับซ้อน ไม่มีการใช้ตราสารหนี้เสมือนทุน |

**หมายเหตุ**: การวิเคราะห์โครงสร้างทางการเงินเป็นเพียงส่วนหนึ่งของการประเมินความเสี่ยงและผลตอบแทนของบริษัท SCC  ควรพิจารณาปัจจัยอื่น ๆ เช่น อัตราส่วนทางการเงิน การวิเคราะห์ความเสี่ยงด้านการดำเนินงาน และแนวโน้มอุตสาหกรรมร่วมด้วย
